In [ ]:
!pip3 install av

In [1]:
import json
import boto3
import io
import imageio
import numpy as np
import tqdm
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import av

In [5]:
##### demo #####
runtime = boto3.Session().client(service_name='sagemaker-runtime')
freq = {}
arr = []
output_frames = []
# 6 sec 
# video_name = 'videos/IMG_3553.MOV'
# long 
video_name = 'videos/IMG_3547.MOV' 
#### extracing frames ####
def extract_frames(video_path):
    frames = []
    video = av.open(video_path)
    for frame in video.decode(0):
        yield frame.to_image() 

##### enumerate through video #####
for j, frame in tqdm.tqdm(enumerate(extract_frames(video_name)), desc="Processing frames"):
    if j >= 600:
        break
    # formatting image byte array #
    frame = frame.resize((320,240), Image.ANTIALIAS)
    byteImgIO = io.BytesIO()
    frame.save(byteImgIO, "PNG")
    byteImgIO.seek(0)
    byteImg = byteImgIO.read()
    # send to endpoint #
    response = runtime.invoke_endpoint(
        EndpointName='pytorch-inference-2021-08-04-21-40-58-634',
        ContentType='application/x-image',
        Body=bytearray(byteImg)
    )
    # read response and create frequency #
    res = json.loads(response['Body'].read().decode())
    ## draw gif ##
    d = ImageDraw.Draw(frame)
    d.text(xy=(10, 10), text=res[0], fill=(255, 255, 255))
    output_frames += [frame]
    ## histogram ##
    if res[0] in freq:
        freq[res[0]] += 1
    else:
        freq[res[0]] = 1

print(sorted(freq, key=freq.get, reverse=True)[:5])
print(freq)

Processing frames: 600it [02:09,  4.63it/s]

['TennisSwing', 'WritingOnBoard']
{'TennisSwing': 512, 'WritingOnBoard': 88}


In [ ]:
## save to gif ##
# Processing frames: 1081it [03:50,  4.68it/s]
# Processing frames: 358it [01:13,  4.88it/s]
imageio.mimsave('output.gif', output_frames, fps=60)

In [ ]:
## https://github.com/ipython/ipython/issues/10045
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython import display
from pathlib import Path
gifPath = Path("output.gif")
# Display GIF in Jupyter, CoLab, IPython
with open(gifPath,'rb') as f:
    display.Image(data=f.read(), format='png')

In [ ]:
### backup ###
response = runtime.invoke_endpoint(
    EndpointName='pytorch-inference-2021-08-04-21-40-58-634',
    ContentType='application/x-image',
    Body=bytearray(img)
)